In [14]:
import glob
import os

import datetime as dt
from pathlib import Path


import numpy as np
import pandas as pd


from mintpy.objects import timeseries
from mintpy.utils import ptime, readfile, utils as ut

from matplotlib import pyplot as plt



In [16]:
ARIA_vert_timeseries_file= r'C:\Users\bvarugu\Documents\merged_UP_GNSS_corrected_timeseries.h5';
ARIA_vert_ts_dates = timeseries(ARIA_vert_timeseries_file).get_date_list();
ARIA_vert_ts_atr = timeseries(ARIA_vert_timeseries_file).get_metadata();


In [18]:
Annual_map_name = '20220801_20230801'
def get_annual_map(map_name,ts_file,units='m'):
    dates = map_name.split('_')
    dset1 = readfile.read(ts_file,datasetName=dates[0])[0]
    dset2 = readfile.read(ts_file,datasetName=dates[1])[0]
    diff = dset2-dset1;
    if units == 'ft':
        diff *= 3.28084
    return diff
annual_map = get_annual_map(Annual_map_name,ARIA_vert_timeseries_file,units='ft')

In [20]:
xmin,xmax = 4000,6000;
ymin,ymax = 4000, 6000;
test_annual_map = annual_map[ymin:ymax,xmin:xmax]

In [26]:
from scipy.spatial import distance_matrix

def simple_idw(x, y, z, xi, yi):
    dist = distance_matrix(x,y, xi,yi)

    # In IDW, weights are 1 / distance
    weights = 1.0 / dist

    # Make weights sum to one
    weights /= weights.sum(axis=0)

    # Multiply the weights for each interpolated point by all observed Z-values
    zi = np.dot(weights.T, z)
    return zi
x,y = np.indices(test_annual_map.shape)
zi = simple_idw(x.flatten(), y.flatten(), test_annual_map.flatten(), x.flatten(), y.flatten())

ValueError: not enough values to unpack (expected 2, got 1)

0

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(10,10),sharey=True);
ax1 =axes[0]
ax2= axees[1]

ax1.imshow(test_annual_map,cmap='jet',vmin=-0.32,vmax=0.32,interpolation='nearest');
ax2.imshow(test_interpolated_annual_map,cmap='jet',vmin=-0.32,vmax=0.32,interpolation='nearest');